In [11]:
import sys
import os
from os import listdir
import random
import shutil
from matplotlib import pyplot
from matplotlib.image import imread
import numpy as np
from numpy import asarray
from numpy import save
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet, inception_v3, mobilenet_v2
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K

In [2]:
K.set_learning_phase(1)

In [3]:
img_width, img_height = 224, 224
epochs = 10
batch_size = 30

In [4]:
datasetfolder = '/home/parkar-sl/Desktop/Deep/TF2/datasets/covid19/'

# create data generator
# This involves first defining an instance of the ImageDataGenerator
# that will scale the pixel values to the range of 0-1.
datagen = ImageDataGenerator(rescale=1.0/255.0)

# prepare iterators
train_it = datagen.flow_from_directory(datasetfolder + 'train/',
    class_mode='categorical', batch_size=batch_size, target_size=(224, 224))
test_it = datagen.flow_from_directory(datasetfolder + 'val/',
    class_mode='categorical', batch_size=batch_size, target_size=(224, 224))

n_classes = train_it.num_classes

Found 30 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [13]:
def build_model():
    base_model = densenet.DenseNet121(input_shape=(224, 224, 3),
                      weights='imagenet',
                      include_top=False,
                      pooling='avg')
    for layer in base_model.layers:
        layer.trainable = True

    x = base_model.output
    predictions = Dense(n_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

def build_inceptionv3():
    base_model = inception_v3.InceptionV3(input_shape=(224, 224, 3),
                      weights='imagenet',
                      include_top=False,
                      pooling='avg')
    for layer in base_model.layers:
        layer.trainable = True
        
    x = base_model.output
    predictions = Dense(n_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

def build_mobilenetv2():
    base_model = mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3),
                      weights='imagenet',
                      include_top=False,
                      pooling='avg')
    for layer in base_model.layers:
        layer.trainable = True

    x = base_model.output
    predictions = Dense(n_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

In [26]:
model = build_model()
inceptionv3model = build_inceptionv3()
mobilenetv2model = build_mobilenetv2()

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['acc', 'mse'])
inceptionv3model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['acc', 'mse'])
mobilenetv2model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['acc', 'mse'])


early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
filepathdn = "covid19_densenet_{epoch:02d}_{val_acc:.2f}.hdf5"
filepathin = "covid19_inception_{epoch:02d}_{val_acc:.2f}.hdf5"
filepathmn = "covid19_mobilenet_{epoch:02d}_{val_acc:.2f}.hdf5"

checkpointdn = ModelCheckpoint(filepathdn, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
checkpointin = ModelCheckpoint(filepathin, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
checkpointmn = ModelCheckpoint(filepathmn, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks_list_dn = [early_stop, reduce_lr, checkpointdn]
callbacks_list_in = [early_stop, reduce_lr, checkpointin]
callbacks_list_mn = [early_stop, reduce_lr, checkpointmn]

In [27]:
model_history = model.fit_generator(train_it,
                                    epochs=epochs,
                                    validation_data=test_it,
                                    validation_steps=test_it.samples // batch_size,
                                    callbacks=callbacks_list_dn)

Epoch 1/10
1/1 [==============================] - 26s 26s/step - loss: 0.7021 - acc: 0.5667 - mse: 0.2520 - val_loss: 0.5738 - val_acc: 0.7500 - val_mse: 0.1959

Epoch 00001: val_acc improved from -inf to 0.75000, saving model to covid19_densenet_01_0.75.hdf5
Epoch 2/10
1/1 [==============================] - 8s 8s/step - loss: 0.2090 - acc: 1.0000 - mse: 0.0433 - val_loss: 0.3462 - val_acc: 0.9000 - val_mse: 0.0994

Epoch 00002: val_acc improved from 0.75000 to 0.90000, saving model to covid19_densenet_02_0.90.hdf5
Epoch 3/10
1/1 [==============================] - 8s 8s/step - loss: 0.0569 - acc: 1.0000 - mse: 0.0055 - val_loss: 0.2206 - val_acc: 0.9500 - val_mse: 0.0529

Epoch 00003: val_acc improved from 0.90000 to 0.95000, saving model to covid19_densenet_03_0.95.hdf5
Epoch 4/10
1/1 [==============================] - 8s 8s/step - loss: 0.0206 - acc: 1.0000 - mse: 8.3422e-04 - val_loss: 0.1727 - val_acc: 0.9500 - val_mse: 0.0416

Epoch 00004: val_acc did not improve from 0.95000
Epoc

In [28]:
inceptionv3model_history = inceptionv3model.fit_generator(train_it,
                                    epochs=epochs,
                                    validation_data=test_it,
                                    validation_steps=test_it.samples // batch_size,
                                    callbacks=callbacks_list_in)

Epoch 1/10
1/1 [==============================] - 13s 13s/step - loss: 0.6941 - acc: 0.4667 - mse: 0.2491 - val_loss: 0.6605 - val_acc: 0.5500 - val_mse: 0.2351

Epoch 00001: val_acc improved from -inf to 0.55000, saving model to covid19_inception_01_0.55.hdf5
Epoch 2/10
1/1 [==============================] - 3s 3s/step - loss: 0.4668 - acc: 0.8333 - mse: 0.1432 - val_loss: 0.5840 - val_acc: 0.6000 - val_mse: 0.2015

Epoch 00002: val_acc improved from 0.55000 to 0.60000, saving model to covid19_inception_02_0.60.hdf5
Epoch 3/10
1/1 [==============================] - 3s 3s/step - loss: 0.1574 - acc: 1.0000 - mse: 0.0249 - val_loss: 0.4463 - val_acc: 0.8500 - val_mse: 0.1414

Epoch 00003: val_acc improved from 0.60000 to 0.85000, saving model to covid19_inception_03_0.85.hdf5
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 0.0608 - acc: 1.0000 - mse: 0.0045 - val_loss: 0.3494 - val_acc: 0.8500 - val_mse: 0.1025

Epoch 00004: val_acc did not improve from 0.85000
Epoch

In [ ]:
mobilenetv2model_history2 = mobilenetv2model.fit_generator(train_it,
                                    epochs=epochs,
                                    validation_data=test_it,
                                    validation_steps=test_it.samples // batch_size,
                                    callbacks=callbacks_list_mn)

Epoch 1/10
1/1 [==============================] - 8s 8s/step - loss: 0.7079 - acc: 0.5667 - mse: 0.2551 - val_loss: 0.5295 - val_acc: 0.7500 - val_mse: 0.1791

Epoch 00001: val_acc improved from -inf to 0.75000, saving model to covid19_mobilenet_01_0.75.hdf5
Epoch 2/10
1/1 [==============================] - 3s 3s/step - loss: 0.4114 - acc: 0.9333 - mse: 0.1230 - val_loss: 0.3905 - val_acc: 0.8500 - val_mse: 0.1211

Epoch 00002: val_acc improved from 0.75000 to 0.85000, saving model to covid19_mobilenet_02_0.85.hdf5
Epoch 3/10
1/1 [==============================] - 3s 3s/step - loss: 0.0929 - acc: 1.0000 - mse: 0.0128 - val_loss: 0.3173 - val_acc: 0.8500 - val_mse: 0.0974

Epoch 00003: val_acc did not improve from 0.85000
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 0.0244 - acc: 1.0000 - mse: 0.0011 - val_loss: 0.2993 - val_acc: 0.9000 - val_mse: 0.0930

Epoch 00004: val_acc improved from 0.85000 to 0.90000, saving model to covid19_mobilenet_04_0.90.hdf5
Epoch 5

In [ ]:
print("model history: ", model_history.history.keys())

plt.figure(0)
plt.plot(model_history.history['acc'], 'r')
plt.plot(model_history.history['val_acc'], 'g')
plt.xticks(np.arange(0, 20, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train', 'validation'])

plt.figure(1)
plt.plot(model_history.history['loss'], 'r')
plt.plot(model_history.history['val_loss'], 'g')
plt.xticks(np.arange(0, 20, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train', 'validation'])

plt.figure(2)
plt.plot(model_history.history['mse'], 'r')
plt.plot(model_history.history['val_mse'], 'g')
plt.xticks(np.arange(0, 20, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("MSE")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train', 'validation'])

plt.show()

In [ ]:
print("model history: ", inceptionv3model_history.history.keys())

plt.figure(0)
plt.plot(inceptionv3model_history.history['acc'], 'r')
plt.plot(inceptionv3model_history.history['val_acc'], 'g')
plt.xticks(np.arange(0, 10, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train', 'validation'])

plt.figure(1)
plt.plot(inceptionv3model_history.history['loss'], 'r')
plt.plot(inceptionv3model_history.history['val_loss'], 'g')
plt.xticks(np.arange(0, 10, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train', 'validation'])

plt.figure(2)
plt.plot(inceptionv3model_history.history['mse'], 'r')
plt.plot(inceptionv3model_history.history['val_mse'], 'g')
plt.xticks(np.arange(0, 10, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("MSE")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train', 'validation'])

plt.show()

In [ ]:
print("model history: ", mobilenetv2model_history2.history.keys())

plt.figure(0)
plt.plot(mobilenetv2model_history.history['acc'], 'r')
plt.plot(mobilenetv2model_history.history['val_acc'], 'g')
plt.xticks(np.arange(0, 10, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train', 'validation'])

plt.figure(1)
plt.plot(mobilenetv2model_history.history['loss'], 'r')
plt.plot(mobilenetv2model_history.history['val_loss'], 'g')
plt.xticks(np.arange(0, 10, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train', 'validation'])

plt.figure(2)
plt.plot(mobilenetv2model_history.history['mse'], 'r')
plt.plot(mobilenetv2model_history.history['val_mse'], 'g')
plt.xticks(np.arange(0, 10, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("MSE")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train', 'validation'])

plt.show()

In [ ]:
import yagmail

In [ ]:
print("model history: ", model_history.history.keys())

plt.figure(0)
plt.plot(model_history.history['acc'], 'r')
plt.plot(model_history.history['val_acc'], 'g')
plt.xticks(np.arange(0, 16, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train', 'validation'])
plt.savefig('acc.png')

plt.figure(1)
plt.plot(model_history.history['loss'], 'r')
plt.plot(model_history.history['val_loss'], 'g')
plt.xticks(np.arange(0, 16, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train', 'validation'])
plt.savefig('loss.png')

plt.figure(2)
plt.plot(model_history.history['mse'], 'r')
plt.plot(model_history.history['val_mse'], 'g')
plt.xticks(np.arange(0, 16, 1.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("MSE")
plt.title("Training MSE vs Validation MSE")
plt.legend(['train', 'validation'])
plt.savefig('mse.png')

plt.show()

In [ ]:
yag = yagmail.SMTP("shblrb", "@SaincTelette70")
contents = ['Hello,\n', '\nTraining has ended successfully! Here are the results:', 
            str(model_history.history), '\nBest regards, \nYour Laptop', 
            yagmail.inline("acc.png"),
            yagmail.inline("loss.png"),
            yagmail.inline("mse.png")]
yag.send('sohaib-l@hotmail.fr', 'Training Ended - NTU49_revised', contents)